# Obsolete - do not use

In [ ]:
import os
import shutil
import sys
import pyspark
from pyspark.sql import SparkSession

In [ ]:
import logging
root_logger = logging.getLogger()
root_logger.handlers = []
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO, stream=sys.stdout)

In [ ]:
pyspark.__version__

In [ ]:
def get_data_path(
    base_dir,
    data_file_prefix,
    size_multiplier,
    partitions,
    stripe_size_MiB,
    compression,
    file_format,
    batch,
    ):
    
    basename = '%s-%0.2fx-%dp-%dMiB-batch%d-%s.%s' % (
        data_file_prefix, size_multiplier, partitions, stripe_size_MiB, batch, compression, file_format)
    return os.path.join(base_dir, basename)

In [ ]:
def get_schema_ddl():
    dtypes = [
        ("loan_id", "bigint"),
        ("monthly_reporting_period", "date"),
        ("servicer", "string"),
        ("interest_rate", "double"),
        ("current_actual_upb", "double"),
        ("loan_age", "double"),
        ("remaining_months_to_legal_maturity", "double"),
        ("adj_remaining_months_to_maturity", "double"),
        ("maturity_date", "string"),
        ("msa", "double"),
        ("current_loan_delinquency_status", "int"),
        ("mod_flag", "string"),
        ("zero_balance_code", "string"),
        ("zero_balance_effective_date", "date"),
        ("last_paid_installment_date", "date"),
        ("foreclosed_after", "date"),
        ("disposition_date", "date"),
        ("foreclosure_costs", "double"),
        ("prop_preservation_and_repair_costs", "double"),
        ("asset_recovery_costs", "double"),
        ("misc_holding_expenses", "double"),
        ("holding_taxes", "double"),
        ("net_sale_proceeds", "double"),
        ("credit_enhancement_proceeds", "double"),
        ("repurchase_make_whole_proceeds", "double"),
        ("other_foreclosure_proceeds", "double"),
        ("non_interest_bearing_upb", "double"),
        ("principal_forgiveness_upb", "double"),
        ("repurchase_make_whole_proceeds_flag", "string"),
        ("foreclosure_principal_write_off_amount", "double"),
        ("servicing_activity_indicator", "string")
    ]
    schema = ','.join([' '.join(col) for col in dtypes])
    return schema

In [ ]:
def get_mortgage_dataframe(
    base_dir='/mnt/isilon1/data/mortgage',
    input_file='perf/Performance_*.txt',
    size_multiplier=1.0,
    partitions=48,
    stripe_size_MiB=64,
    ):
    
    spark = (SparkSession
         .builder
         .config('spark.driver.memory', '1000g')
         .config('hive.exec.orc.default.stripe.size', stripe_size_MiB*1024*1024)
         .getOrCreate()
         )
    input_path = os.path.join(base_dir, input_file)
    schema = get_schema_ddl()
    df = spark.read.load(input_path, format='csv', sep='|', schema=schema, header=False, dateFormat='MM/dd/yyyy')
    if size_multiplier != 1.0:
        df = df.sample(True, size_multiplier, seed=7)
    if partitions:
        df = df.repartition(partitions)
    return df

In [ ]:
def write_mortgage_dataframe(
    df,
    base_dir='/mnt/isilon1/data/mortgage',
    data_file_prefix='perf-from-spark',
    size_multiplier=1.0,
    partitions=48,
    stripe_size_MiB=64,
    compression='snappy',
    file_format='orc',
    batch=0,
    ):
    
    output_path = get_data_path(base_dir, data_file_prefix, size_multiplier, partitions, stripe_size_MiB, compression, file_format, batch)
    logging.info('write_mortgage_dataframe: output_path=%s' % output_path)
#     if size_multiplier != 1.0:
#         df = df.sample(True, size_multiplier, seed=7+batch)
    if os.path.exists(output_path): shutil.rmtree(output_path)
    (df.write
     .format(file_format)
     .option('compression',compression)
     .save(output_path))
    logging.info('write_mortgage_dataframe: done.')    

In [ ]:
partitions = 48
stripe_size_MiB = 2048
size_multiplier = 3.0
df = get_mortgage_dataframe(
        size_multiplier=size_multiplier,
        partitions=partitions,
        stripe_size_MiB=stripe_size_MiB,
        )
df = df.cache()

In [ ]:
%%time
for compression in ['snappy']:
    for file_format in ['orc']:
        for batch in range(100):
            write_mortgage_dataframe(
                df,
                size_multiplier=size_multiplier,
                partitions=partitions,
                stripe_size_MiB=stripe_size_MiB,
                compression=compression,
                file_format=file_format,
                batch=batch,
        )